In [1]:
import os
import sys
import warnings

# set environment path
sys.path.append("../..")

import numpy as np
import pandas as pd

from gensim.models.doc2vec import Doc2Vec
from sqlalchemy import create_engine
from tqdm import tqdm

from src.models import algorithms, evaluate_model
from src.utilities import utilities

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data and Models

In [2]:
# set path to where repo is located
REPO_PATH = os.path.join("..", "..")

# setting path
DATA_PATH = os.path.join(REPO_PATH, "data", "evaluation")
D2V_PATH = os.path.join(REPO_PATH, "models", "d2v")

# category to train
CATEGORY = "Pet_Supplies"

# mod-ecf training parameters, training epochs, learning, and regularisation rate
PARAMS = {"num_epochs": 5,
          "learning_rate": 0.005,
          "beta": 0.1}

# reproducibility checks
SEED = 42
np.random.seed(42)

# load train/test dataset and d2v trained model
train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")
d2v = Doc2Vec.load(f"{D2V_PATH}/{CATEGORY}_user_item_50_10_d2v.model")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A14CK12J7C7JRK,3.0,I purchased the Trilogy with hoping my two cat...,2011-01-12,purchase trilogy hop cat age interested yr old...
1,2,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2CR37UY3VR7BN,4.0,I bought the triliogy and have tested out all ...,2012-12-19,buy triliogy test dvd appear volume receive re...
2,3,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2A4COGL9VW2HY,4.0,My female kitty could care less about these vi...,2011-05-12,female kitty care video care little male dig a...
3,4,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2UBQA85NIGLHA,3.0,"If I had gotten just volume two, I would have ...",2012-03-05,volume star trilogy star read review know vol ...
4,5,B00005MF9U,LitterMaid LM900 Mega Self-Cleaning Litter Box,"['Pet Supplies', 'Cats', 'Litter &amp; Housebr...",A2BH04B9G9LOYA,1.0,"First off, it seems that someone is spamming t...",2006-12-31,spamming review glow reviewer review amazon ba...
68865,111581,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A11J1FHCK5U06J,4.0,Now I know exactly where the trouble spots are...,2014-05-23,know exactly trouble spot sniffing guess invis...
68866,111585,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A18JF0T0GOCORW,4.0,I use this light to help me find stains when I...,2014-05-24,use light help stain carpet clean pre treat ca...
68867,111595,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A3GRPCW9DG427Z,5.0,We are owned by the 3 pickiest pooches in the ...,2013-07-27,pickiest pooch world love fool reject doggie t...
68868,111598,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A2X6TLAX3JEO1A,5.0,My highly allergic white boxer loves these tre...,2014-05-09,highly allergic white boxer love treat meat co...
68869,111602,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A9PG9ODPPP31N,5.0,Works great on my medium sized dog. She has ve...,2014-07-09,work great medium size dog coarse hair work gr...


In [4]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,1,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A39QHP5WLON5HV,5.0,There are usually one or more of my cats watch...,2013-09-14,usually cat watch tv stay trouble dvd play lik...
1,104,B00005MF9V,LitterMaid Universal Cat Privacy Tent (LMT100),"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A366V0GCEPH5CX,5.0,My cats love it and so do I. I no longer have ...,2013-02-02,cat love longer cat litter fly floor litter fl...
2,133,B00005MF9T,LitterMaid LM500 Automated Litter Box,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",ALWWS8QBYN80B,1.0,I have one female cat that weighs under 10 pou...,2004-11-17,female cat weigh pound year old use everclean ...
3,153,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A3PVI3NE7OY1SP,5.0,I love these. They make the clean up so much e...,2013-09-26,love clean easy clean box manually use issue w...
4,154,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A2H83XMHUVDLJY,4.0,"I love this litter box. I do not use the lids,...",2014-06-26,love litter box use lid use receptacle tear cr...
41564,111601,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],AV34KNYW82YSS,4.0,Pulled lots of hair out of my Labs coat. Didn'...,2014-07-18,pulled lot hair labs coat think prove wrong co...
41565,111603,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1YMNTFLNDYQ1F,5.0,I have been trying to find a rubber bristle br...,2014-07-16,try rubber bristle brush persian year lose glo...
41566,111604,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1FQ3HRVXA4A5B,5.0,Great product to use on your pets knowing this...,2014-07-11,great product use pet know gentle rubber damag...
41567,111605,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A3OP6CI0XCRQXO,5.0,I bought a second one because I have two cats ...,2014-07-22,buy second cat american short hair buy brush m...
41568,111606,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A11LC938XF35XN,5.0,Our dogs love getting brushed with this. It m...,2014-07-17,dog love brush massage remove heavy undercoat ...


In [5]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))
print(test_user_history)

                  reviewerID                                  asin
0      A04173782GDZSQ91AJ7OD              [B0090Z9AYS, B00CPDWT2M]
1      A042274212BJJVOBS4Q85              [B005AZ4M3Q, B00771WQIY]
2       A0436342QLT4257JODYJ  [B0018CDR68, B003SJTM8Q, B00474A3DY]
3      A04795073FIBKY8GSLZYI              [B001PKT30M, B005DGI2RY]
4      A06658082A27F4VB5UG8E              [B000TZ1TTM, B0019VUHH0]
...                      ...                                   ...
18993          AZYJE40XW6MFG              [B00HVAKJZS, B00IDZT294]
18994          AZZ56WF4X19G2                          [B004A7X218]
18995          AZZNK89PXD006  [B0002DHV16, B005BP8MQ8, B009RTX4SU]
18996          AZZV9PDNMCOZW              [B007EQL390, B00ISBWVT6]
18997          AZZYW4YOE1B6E  [B0002AQPA2, B0002AQPA2, B0002ARQV4]

[18998 rows x 2 columns]


In [6]:
# testing d2v models
d2v.dv[0]

array([-0.27808702, -0.12123964, -0.01402191, -0.06815875, -0.0564175 ,
        0.04842656,  0.0368541 ,  0.01398593, -0.14648551,  0.08242252,
        0.06371399,  0.15403804,  0.05145778,  0.04584663, -0.06243193,
       -0.08451284,  0.00833067, -0.02114333, -0.08585242,  0.0117121 ,
        0.01590339,  0.00859375, -0.11234739, -0.09751832, -0.00462302,
        0.16073631,  0.03879074, -0.1404444 , -0.07833791, -0.14203373,
        0.01051444,  0.07330766,  0.00680773,  0.10980774,  0.0329658 ,
        0.15197492, -0.09528293, -0.15051496,  0.13257255, -0.00050401,
       -0.04297897,  0.12316997, -0.0282629 , -0.00220049,  0.02751359,
        0.13378084,  0.11880151, -0.06431285, -0.03257735, -0.04988131],
      dtype=float32)

# Generating User/Item Embeddings

In [7]:
user_idx_map, user_vecs, item_idx_map, item_vecs = utilities.generate_user_item_embeddings(train, d2v)

In [8]:
# converting factors into numpy obj
user_factors = user_vecs.to_numpy()
item_factors = item_vecs.to_numpy()

In [9]:
# check user factors
user_factors[0,:]

array([-0.8208622 , -0.13265412,  0.08794311, -0.31527147,  0.05089829,
        0.01806653, -0.01790568,  0.12906374, -0.32686502,  0.34363213,
        0.11507503,  0.19288237,  0.2258099 ,  0.139002  , -0.02829647,
        0.18842314, -0.04987793,  0.06237138, -0.21363561, -0.17762592,
       -0.01040701,  0.21963714, -0.07449435, -0.01158117, -0.08465536,
        0.28415218, -0.14400692, -0.493495  , -0.26090425, -0.24364568,
        0.28065497, -0.07475447,  0.03166582,  0.06282842,  0.05320998,
        0.53347766, -0.19234563,  0.00528651,  0.4275993 ,  0.3116027 ,
       -0.08872687,  0.17197491, -0.19117323, -0.19778593, -0.1686984 ,
        0.27757648,  0.18943855, -0.05875679, -0.06944423,  0.0362678 ],
      dtype=float32)

In [10]:
# check item factors
item_factors[0,:]

array([-0.27808702, -0.12123964, -0.01402191, -0.06815875, -0.0564175 ,
        0.04842656,  0.0368541 ,  0.01398593, -0.14648551,  0.08242252,
        0.06371399,  0.15403804,  0.05145778,  0.04584663, -0.06243193,
       -0.08451284,  0.00833067, -0.02114333, -0.08585242,  0.0117121 ,
        0.01590339,  0.00859375, -0.11234739, -0.09751832, -0.00462302,
        0.16073631,  0.03879074, -0.1404444 , -0.07833791, -0.14203373,
        0.01051444,  0.07330766,  0.00680773,  0.10980774,  0.0329658 ,
        0.15197492, -0.09528293, -0.15051496,  0.13257255, -0.00050401,
       -0.04297897,  0.12316997, -0.0282629 , -0.00220049,  0.02751359,
        0.13378084,  0.11880151, -0.06431285, -0.03257735, -0.04988131],
      dtype=float32)

# Generate N-Recommendations = {5, 10, 15, 20}

## Instantiate Pre-Initialised Matrix Factorization (Paragraph Vector)

In [11]:
# instantiating mod-ecf
mod_ecf = algorithms.PreInitialisedMF(user_map=user_idx_map,
                                      item_map=item_idx_map,
                                      user_factor=user_factors,
                                      item_factor=item_factors,
                                      num_factors=50,
                                      **PARAMS)

In [12]:
%%time
# fitting to training data
mod_ecf.fit(train, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
CPU times: user 2min 55s, sys: 897 ms, total: 2min 56s
Wall time: 2min 57s


In [13]:
%%time
# generate candidate items for user to predict rating
testset = mod_ecf.trainset.build_anti_testset()

CPU times: user 48.6 s, sys: 2.32 s, total: 50.9 s
Wall time: 51.6 s


In [14]:
%%time
# predict ratings for all pairs (u, i) that are NOT in the training set
candidate_items = mod_ecf.test(testset, verbose=False)

CPU times: user 11min 35s, sys: 3min 48s, total: 15min 23s
Wall time: 17min 6s


## Loop through N = {5, 10, 15, 20}

For each top-N setting, we will generate candidates items up to *N*-items and run metrics evaluation of `Recall@N` and `Novelty@N` on all users.

In [15]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [16]:
%%time
n_recommendations = {}
for n in [5, 10, 15, 20]:
    # retrieve the top-n items based on similarities
    top_ns = mod_ecf.get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name="MOD-ECF",
                                                    top_ns=top_ns,
                                                    user_rating_history=test_user_history, 
                                                    item_popularity=item_popularity, 
                                                    n=n,
                                                    mf_based=True))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The MOD-ECF has an average recall@5: 0.00439, average novelty@5: 0.95870
The MOD-ECF has an average recall@10: 0.00840, average novelty@10: 0.96224
The MOD-ECF has an average recall@15: 0.01274, average novelty@15: 0.96423
The MOD-ECF has an average recall@20: 0.01571, average novelty@20: 0.96555
CPU times: user 6min 35s, sys: 3min 34s, total: 10min 10s
Wall time: 13min 34s


# Evaluate N-Recommendations

For each top-N recommendation list, we pick a randomly sampled user to look at their *N*-number of recommendations based on their purchase history.

## N=5

In [17]:
top_ns_05 = n_recommendations[5][0]
utilities.retrieve_recommendations(train, top_ns_05, mf_based=True)

For user: A3T87QAUUPTMZK:
Purchase History:
             asin                                              title
43366  B0010OSIHW  Zoo Med Eco Earth Compressed Coconut Fiber Sub...
45100  B00167VVP4   Zoo Med Eco Earth Loose Coconut Fiber Substra...
46998  B0019IJXD2       Zoo Med Reptile Fogger Terrarium Humidifier 

Recommending:

         asin                                              title
0  B005HJYXNO         AquaTop In-Line UV Sterilizer 10W - IL10UV
1  B005VFLAG2       Finnex External Refugium Breeder Hang-On Box
2  B000MPR2GI       Good Pet Stuff Company Hidden Cat Litter Box
3  B0002DJONY                            Vittles Vault Stackable
4  B001E530FM  Purina Tidy Cats BREEZE Litter System Cat Pad ...


## N=10

In [18]:
top_ns_10 = n_recommendations[10][0]
utilities.retrieve_recommendations(train, top_ns_10, mf_based=True)

For user: A1XEZIHQIUAOR1:
Purchase History:
             asin                                              title
1487   B000084E6V                      Nylabone Dental Dinosaur Chew
1727   B000084E6V                      Nylabone Dental Dinosaur Chew
28789  B0006VMN4O  Pioneer Pet SmartCat Peek-A-Prize Toy Box with...
37658  B000JZ1WSU                     SmartCat 3836 Tick Tock Teaser
42603  B000XZDV44  Hill'S Science Diet Kitten Savory Salmon Entre...
43867  B0012KCUOG  Whisker Lickin'S Soft &amp; Delicious Chicken ...

Recommending:

         asin                                              title
0  B000084E7Y                  Multipet Duckworth Duck Large 13"
1  B00028ZLVS  Sergeants - Pet Specialty Division Petrodex Do...
2  B0002AQ81I                        Penn Plax Knot Dog Rope Toy
3  B0002ASGYU  LPVLUX Deshedding Gloves 1 Pair - Dog Grooming...
4  B0002ASM94  Nylabone Dura Chew Pooch Pacifier Chew Bone Va...
5  B0002ASN5W                    Original Knot Bone Dog Chew Toy
6 

## N=15

In [19]:
top_ns_15 = n_recommendations[15][0]
utilities.retrieve_recommendations(train, top_ns_15, mf_based=True)

For user: A89LQAXW1IY6S:
Purchase History:
             asin                                              title
12819  B0002ARP2O  Marshall Ferret Deluxe Leisure Lounge, Pattern...
12825  B0002ARP2O  Marshall Ferret Deluxe Leisure Lounge, Pattern...
38837  B000MD3NLS  MidWest Homes for Pets Snap'y Fit Stainless St...
42683  B000Y8UNAU  Pro Select Fleece Cat Perch Covers - Comfortab...
46465  B0018CJZ32                         SmartCat Corner Litter Box

Recommending:

          asin                                              title
0   B0002DGVY4  Herm Sprenger Pet Supply Imports Chrome Plated...
1   B0002Z7VAK  EHEIM Substrat Pro Biological Filter Media (Si...
2   B0002Z9A4U  Muttluks Original All-Weather IB to XXL Dog Bo...
3   B00061MWJA  Oster Professional Pet Grooming Undercoat Rake...
4   B00061V0M0      Vo-Toys 1208 Ferret Cat Toy Cat &amp; Dog Toy
5   B000633XBE  Rinse Ace Deluxe Pet Shower Sprayer with 8 foo...
6   B000634MXM                           Snoozer Lookout Car Seat

## N=20

In [20]:
top_ns_20 = n_recommendations[20][0]
utilities.retrieve_recommendations(train, top_ns_20, mf_based=True)

For user: A3C2ECIXEQ0YFQ:
Purchase History:
             asin                                              title
30751  B0009YS4P0  Nutri-Vet Hip &amp; Joint Extra Strength Chewa...
37067  B000IBRI2Y              Dog Dazer II Ultrasonic Dog Deterrent
40175  B000OV4VAU                        Nutri-Vet Alaska Salmon Oil
53623  B0029NQTI8  Pedigree Choice Cuts Variety Pack Lamb/Vegetab...

Recommending:

          asin                                              title
0   B001P220TQ   Hikari Usa Inc AHK42553 Saki Goldfish 7.0 -Ounce
1   B000VK6N14  Castor &amp; Pollux Natural Ultramix Dry Cat F...
2   B001BCOZ9S  Castor &amp; Pollux Natural Ultramix Dry Puppy...
3   B0018CG2DI  Chicken Soup For The Dog Lover'S Soul Canned F...
4   B003R0MFX0  Tiki Cat Napili Wild Salmon And Chicken In Chi...
5   B0089A62MS                       Merrick Classic Real Chicken
6   B000A6UF4U     Gamma2 Vittles Vault Plus for Pet Food Storage
7   B0017JGAB6  Iceland Pure Unscented Pharmaceutical Grade Sa...
8

# Cross-Analysis for Cold-Start Users (<= 2 Purchased Items)

For each top-N setting, we will generate candidates items up to *N*-items and run metrics evaluation of `Recall@N` and `Novelty@N` on cold-start users (e.g., users who purchased two or less items based on items per user in the training set).

In [21]:
cold_start_users = utilities.generate_cold_start_users(train)

In [22]:
for n in tuple(zip([5, 10, 15, 20], [top_ns_05, top_ns_10, top_ns_15, top_ns_20])):
    cold_start_top_ns = dict(filter(lambda x: x[0] in cold_start_users, n[1].items()))
    # evaluate how well the recommended items predicted the future purchases
    # on cold start users
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name="MOD-ECF",
                                                    top_ns=cold_start_top_ns,
                                                    user_rating_history=test_user_history, 
                                                    item_popularity=item_popularity, 
                                                    n=n[0],
                                                    mf_based=True))

The MOD-ECF has an average recall@5: 0.00524, average novelty@5: 0.96737
The MOD-ECF has an average recall@10: 0.01000, average novelty@10: 0.97012
The MOD-ECF has an average recall@15: 0.01493, average novelty@15: 0.97165
The MOD-ECF has an average recall@20: 0.01788, average novelty@20: 0.97258


# Generating Recommended Items DataFrame

In [23]:
max_recommendations = (utilities
                       .generate_recommendations_df(train=train,
                                                    n_recommendations=n_recommendations,
                                                    algo_name="MOD-ECF",
                                                    mf_based=True,
                                                    max_recommended=20))

In [24]:
max_recommendations

,reviewerID,item_rank,asin,algorithm,title
0,A04173782GDZSQ91AJ7OD,0,B001E530FM,MOD-ECF,Purina Tidy Cats BREEZE Litter System Cat Pad ...
1,A04173782GDZSQ91AJ7OD,1,B000MPR2GI,MOD-ECF,Good Pet Stuff Company Hidden Cat Litter Box
2,A04173782GDZSQ91AJ7OD,2,B001U8L8NO,MOD-ECF,Van Ness Odor Control Extra Giant Translucent...
3,A04173782GDZSQ91AJ7OD,3,B002CYNMTS,MOD-ECF,Tidy Cats Cat Litter Pan
4,A04173782GDZSQ91AJ7OD,4,B0002DJONY,MOD-ECF,Vittles Vault Stackable
...,...,...,...,...,...
381155,AZZYW4YOE1B6E,15,B001J8H1E0,MOD-ECF,PLATO Dog Treats - Organic Chicken Real Strips
381156,AZZYW4YOE1B6E,16,B000O5DI1E,MOD-ECF,Beef Pill Pockets Large 7.9oz
381157,AZZYW4YOE1B6E,17,B000OUOYH6,MOD-ECF,Pet Botanics Training Reward
381158,AZZYW4YOE1B6E,18,B000MCXXKU,MOD-ECF,"Bio Spot Spot On for Dogs 16-30 lbs., 6 Month ..."


# Store in `SQLite` DB

In [25]:
# engine = create_engine("sqlite:///recommender.db", echo=True)

In [26]:
# max_recommendations.to_sql(f"{CATEGORY}", con=engine, if_exists="append")